In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
import statsmodels

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/Mastertabelle_filtered.xlsx'
df = pd.read_excel(data_path)

In [ ]:
df = df[df.Comment != 'Reanimation']

In [ ]:
# encode eineteilung
df['Einteilung_encoded'] = df['Einteilung (Hauptdiagnose)'].map({'Krankheit': 0, 'Unfall': 1})
df['Sverweis_Geschlecht_encoded'] = df['Sverweis_Geschlecht'].map({'männlich': 0, 'weiblich': 1})
df.loc[(df['Einteilung (Hauptdiagnose)'] == 'Unfall') & (
            df['Kategorie (Hauptdiagnose)'] == 'Respiratorisch'), 'Kategorie (Hauptdiagnose)'] = 'Trauma_Respiratorisch'
# numerical encoding for Eineteilung and Kategorie
df['Kategorie_encoded'] = df['Kategorie (Hauptdiagnose)'].map({'Zentralnervensystem': 0,
                                                                         'Krampfanfall': 1,
                                                                         'Herz-Kreislauf': 2,
                                                                         'Intoxikation': 3,
                                                                         'Respiratorisch': 4,
                                                                         'SHT': 5,
                                                                         'Polytrauma': 6,
                                                                         'Verbrennung': 7,
                                                                         'Trauma_Respiratorisch': 8})

In [ ]:
df.head()

univariate analysis

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = df[['GCS (Erstbefund)', 'Sverweis_Geschlecht_encoded']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Sverweis_Geschlecht_encoded']], distr='probit')

In [ ]:
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

multivariate analysis

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = df[['GCS (Erstbefund)', 'Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']], distr='probit')

In [ ]:
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = df[['GCS (Erstbefund)', 'Kategorie_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'],
                            temp_df[['Kategorie_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']],
                            distr='probit')
res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()

plot

In [ ]:
df['Category'] = df['Einteilung (Hauptdiagnose)'].map({'Krankheit': 'Non trauma', 'Unfall': 'Trauma'})

In [ ]:
# gcs vs sex
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='Sverweis_Geschlecht', y='GCS (Erstbefund)', hue='Category', data=df[~df.Category.isna()], palette='Set3', ax=ax)

ax.set_ylim(3, 17)
plt.ylabel('GCS')
# don't allow y_ticks > 15
plt.yticks(range(3, 16, 2))

# set x ticks
plt.xticks(range(2), ['Female', 'Male'])

ax.set_xlabel('')

# set legend location upper left
plt.legend(loc='upper left')

In [ ]:
# fig.savefig('/Users/jk1/Downloads/gcs_vs_sex.png', dpi=600, bbox_inches='tight')